In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os
from mne.datasets import visual_92_categories
import pandas as pd
import mne

In [2]:
with open('data/visual_stimuli/visual_stimuli_ica_preprocessed.pickle', 'rb') as f:
    data = pickle.load(f)

In [9]:
data.shape

(2758, 306, 601)

In [16]:
list(np.linspace(0, 92, 3, dtype=int)[:2])

[0, 46]

In [3]:
print(data.mean(), data.std())

-9.876539e-15 5.542058e-12


In [8]:
# normalize data between 0 and 1
data = torch.Tensor((data - np.min(data)) / (np.max(data) - np.min(data)))

print(data.mean(), data.std())

tensor(0.5779) tensor(0.0141)


In [9]:
data.shape

torch.Size([2758, 306, 601])

In [70]:
# save the labeled target class for each sample

data_path = visual_92_categories.data_path()

# Define stimulus - trigger mapping
fname = data_path / 'visual_stimuli.csv'
conds = pd.read_csv(fname)

conds_tensor = torch.Tensor(np.array(conds[['human', 'face', 'animal', 'natural']]))

In [71]:
conds

,trigger,condition,human,face,animal,natural
0,0,human bodypart,1,0,1,1
1,1,human bodypart,1,0,1,1
2,2,human bodypart,1,0,1,1
3,3,human bodypart,1,0,1,1
4,4,human bodypart,1,0,1,1
...,...,...,...,...,...,...
87,87,artificial inanimate,0,0,0,0
88,88,artificial inanimate,0,0,0,0
89,89,artificial inanimate,0,0,0,0
90,90,artificial inanimate,0,0,0,0


In [72]:
import pickle

with open('./data/visual_stimuli/events_label.pkl', 'rb') as f:
    labels = pickle.load(f)

In [67]:
target_class = torch.zeros(labels.shape[0], 4, dtype=torch.long)

for i, label in enumerate(labels):
    target_class[i] = conds_tensor[int(label.item())]

target_class.shape

import pickle

with open('./data/visual_stimuli/target_class.pickle', 'wb') as f:
    pickle.dump(target_class, f, protocol=pickle.HIGHEST_PROTOCOL)


In [68]:
# split data into train and test
s_train = int(number_samples*0.8)
s_test = number_samples - s_train

In [69]:
s_test

552

In [71]:
s_train

Load and train the transformer encoder model to perform classifiction of the visual sitmuli given the eeg/meg preprocess data responses.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os
from mne.datasets import visual_92_categories
import pandas as pd

with open('./data/visual_stimuli/target_class.pickle', 'rb') as f:
    target_class = pickle.load(f)
# with open('./data/visual_stimuli/events_label.pkl', 'rb') as f:
#     target_class = pickle.load(f)
with open('./data/visual_stimuli/visual_stimuli_ica_preprocessed.pickle', 'rb') as f:
    data_exp = pickle.load(f)

# normalize data between 0 and 1
data_exp = torch.Tensor((data_exp - np.min(data_exp)) / (np.max(data_exp) - np.min(data_exp)))

print(data_exp.mean(), data_exp.std())

# prepare data for training
number_samples = 2758

# add cls token
cls_ = torch.zeros(number_samples, 1, data_exp[0].shape[1])

data_exp = torch.cat((cls_, data_exp), dim=1)

tensor(0.5779) tensor(0.0141)


In [2]:
# split data into train and test
s_train = int(number_samples*0.8)
s_test = number_samples - s_train

batch = 64

generator = torch.Generator().manual_seed(42)

data = torch.utils.data.TensorDataset(data_exp, target_class.type(torch.LongTensor))
training_samples, validation_samples = torch.utils.data.random_split(data, [s_train, s_test], generator=generator)

train = torch.utils.data.DataLoader(training_samples, batch_size=batch, shuffle=True)
val = torch.utils.data.DataLoader(validation_samples, batch_size=batch, shuffle=False)

In [5]:
from module._module import *

net = Net(dim_emb=601, dim_emb_transformer=64,number_classes=4, nhead=2, dim_feedforward=32).cuda()

In [4]:
net(data_exp[:64].cuda()).shape

torch.Size([64, 91])

In [7]:
from tranformer import training
from torch import optim

training(net, train, optimizer = optim.AdamW(net.parameters(), lr=0.001), batch_size=batch, epochs=100)

100%|██████████| 9/9 [00:00<00:00,  9.84it/s]


epoch: 1, loss: 1.075, accuracy: 0.544


100%|██████████| 9/9 [00:00<00:00, 12.54it/s]


epoch: 2, loss: 1.078, accuracy: 0.544


100%|██████████| 9/9 [00:00<00:00, 12.52it/s]


...

epoch: 98, loss: 1.076, accuracy: 0.544


100%|██████████| 9/9 [00:00<00:00, 13.33it/s]


epoch: 99, loss: 1.076, accuracy: 0.544


100%|██████████| 9/9 [00:00<00:00, 13.53it/s]

epoch: 100, loss: 1.076, accuracy: 0.544


In [3]:
from tranformer import Netv2

net = Netv2(dim_emb=601, dim_emb_transformer=295,number_classes=4, nhead=1, dim_feedforward=16).cuda()

from tranformer import training
from torch import optim

training(net, train, optimizer = optim.AdamW(net.parameters(), lr=0.001), batch_size=batch, epochs=10)

cuda:0


  0%|          | 0/35 [00:01<?, ?it/s]

torch.Size([295, 295])
torch.Size([295, 4])


RuntimeError: The size of tensor a (295) must match the size of tensor b (64) at non-singleton dimension 0

In [6]:
from tranformer import test

test(net, val, batch_size=batch)


100%|██████████| 5/5 [00:00<00:00, 12.89it/s]

loss: 5.720, accuracy: 0.009


In [52]:
outputs = torch.nn.Softmax()(torch.randn((10,4)))
label = torch.randint(0, 2, (10,4))

(outputs == label).sum().item()/(10*4)

/tmp/ipykernel_1003/3095245632.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  outputs = torch.nn.Softmax()(torch.randn((10,4)))


RuntimeError: The size of tensor a (10) must match the size of tensor b (4) at non-singleton dimension 1

In [5]:
out = net(data_exp[:200].cuda())

In [12]:
class MLP(nn.Module):
    def __init__(self, dim_emb, dim_feedforward: int = 10, number_classes: int = 92, dropout: float =0.1, activation = nn.GELU):
        super(MLP, self).__init__()
        
        # Implementation of Feedforward model
        self.linear1 = nn.Linear(dim_emb, dim_feedforward)
        self.linear2 = nn.Linear(dim_feedforward, number_classes)
        
        self.dropout = nn.Dropout(dropout)
        self.activation = activation()

    def forward(self, x):
        
        x = self.linear2(self.dropout(self.activation(self.linear1(x))))    
            
        return x
    

nn.Linear((601**2), 10)(torch.randn((64,601,601)).reshape(64,601**2)).shape

torch.Size([64, 10])

In [9]:
out = torch.randn((295,1))


In [27]:
nn.AdaptiveMaxPool1d(out.shape[0]//4)(out.permute(1,0)).shape

torch.Size([1, 73])

nn.Conv2d(in_channels=3, out_channels=3, kernel_size=3, groups=3)

In [12]:
nn.Conv2d(in_channels=607, out_channels=1, kernel_size=(307,1), stride=1, dilation=1)(
    torch.randn((64,307,607)).permute(2,1,0)
).squeeze(0).permute(1,0).shape

torch.Size([64, 1])

In [11]:
torch.randn((64,307,607)).permute(2,1,0).shape

torch.Size([607, 307, 64])